In [1]:
!pip install psycopg2-binary
### boto3 is optional, but recommended to leverage the AWS Secrets Manager storing the credentials  Establishing a Redshift Connection
!pip install boto3

    100% |████████████████████████████████| 3.0MB 451kB/s eta 0:00:01
You are using pip version 9.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.1, however version 21.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# Put the ARN of your AWS Secrets Manager secret for your redshift cluster here:
secret_arn="arn:aws:secretsmanager:us-west-2:123456789012:secret:aes128-1a2b3c"
# This will get the secret from AWS Secrets Manager. 
import boto3
import json
session = boto3.session.Session()
client = session.client(
    service_name='secretsmanager'
)
get_secret_value_response = client.get_secret_value(
    SecretId=secret_arn
)

if 'SecretString' in get_secret_value_response:
    connection_info = json.loads(get_secret_value_response['SecretString'])
else:
    print("ERROR: no secret data found")
# Sanity check for credentials
expected_keys = set(['user', 'password', 'host', 'database', 'port'])
if not expected_keys.issubset(connection_info.keys()):
    print("Expected values for ",expected_keys)
    print("Received values for ",set(connection_info.keys()))
    print("Please adjust query or assignment as required!")

# jdbc:redshift://HOST:PORT/DBNAME
import time
import psycopg2

con=psycopg2.connect(
    dbname   = connection_info["database"], 
    host     = connection_info["host"], 
    port     = connection_info["port"], 
    user     = connection_info["user"], 
    password = connection_info["password"]
)


In [ ]:
cur = con.cursor()
cur.execute("SELECT sysdate")
res = cur.fetchall()
print(res)
cur.close()
